In [2]:
! pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle &> /dev/null
!cp kaggle.json ~/.kaggle/ &> /dev/null
!chmod 600 ~/.kaggle/kaggle.json &> /dev/null
!kaggle datasets download -d surajghuwalewala/ham1000-segmentation-and-classification &> /dev/null
!unzip ham1000-segmentation-and-classification.zip &> /dev/null

Saving kaggle.json to kaggle.json


In [53]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchvision import transforms
import shutil

In [5]:
# csv file -> labels
# 2 folders , 1 for images 1 for masks

# image_folder = "/content/images"
# mask_folder = "/content/masks"
csv_path = "/content/GroundTruth.csv"

# output_base = "/content/ham1000_data"
# train_dir = os.path.join(output_base, "train")
# val_dir = os.path.join(output_base, "val")
# test_dir = os.path.join(output_base, "test")


In [45]:
data_table = pd.read_csv(csv_path)
data_table.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# creating a normal encoding for all the columns for straitfying the dataset
enc_label = pd.DataFrame()
labels = data_table.drop(["image"], axis=1)
enc_label['label'] = labels.idxmax(axis=1)
enc_label['label'] = enc_label['label'].str.replace('MEL', '1')
enc_label['label'] = enc_label['label'].str.replace('NV', '2')
enc_label['label'] = enc_label['label'].str.replace('BCC', '3')
enc_label['label'] = enc_label['label'].str.replace('AKIEC', '4')
enc_label['label'] = enc_label['label'].str.replace('BKL', '5')
enc_label['label'] = enc_label['label'].str.replace('DF', '6')
enc_label['label'] = enc_label['label'].str.replace('VASC', '7')
enc_label.head()

,label
0,2
1,2
2,2
3,2
4,1


In [47]:
data_table = pd.concat([data_table, enc_label], axis=1)
data_table.head()


,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,label
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [52]:
train_df, x_df = train_test_split(data_table, train_size=0.7, stratify=data_table['label'])
val_df, test_df = train_test_split(x_df, train_size=0.5, stratify=x_df['label'])


In [ ]:
 # data preprocessing of images before splitting into train,test,valid
 # dtaa leakage is probably the background of the images being more pink-ish in some, so must adjust coloring

transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.RandomGrayscale(p=0.1)
 ])
